In [4]:
# Note: Before using any library install it first - !pip install "your library name"

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import warnings
# from keras.preprocessing.image import load_img
# warnings.filterwarnings('ignore')
# import os
# import tqdm
# import random
# import tensorflow as tf
# from tensorflow.keras.models import load_model

In [ ]:
# PetImages directory to collect file paths and assign labels for cats (0) and dogs (1)
input_path = []
label = []

for i in os.listdir("PetImages"):
  for path in os.listdir("PetImages/"+i):
    if i == "Cat":
      label.append(0)
    else:
      label.append(1)
    input_path.append(os.path.join("PetImages",i,path))
print(input_path[0],label[0])

In [ ]:
len(label)
len(input_path)

In [ ]:
df = pd.DataFrame()
df['images'] = input_path
df['label'] = label
df = df.sample(frac=1).reset_index(drop=True)
df.head()

In [ ]:
plt.figure(figsize=(25,25))
temp = df[df['label']==1]['images']
start = random.randint(0,len(temp))
files = temp[start:start+25]

for index, file in enumerate(files):
  plt.subplot(5,5,index+1)
  img = load_img(file)
  img = np.array(img)
  plt.imshow(img)
  plt.title("Dogs")
  plt.axis("off")

In [ ]:
plt.figure(figsize=(25,25))
temp = df[df['label']==0]['images']
start = random.randint(0,len(temp))
files = temp[start:start+25]

for index, file in enumerate(files):
  plt.subplot(5,5,index+1)
  img = load_img(file)
  img = np.array(img)
  plt.imshow(img)
  plt.title("cats")
  plt.axis("off")

In [ ]:
# delete db files
for i in df['images']:
  if '.jpg' not in i:
    print(i)

In [ ]:
# lets delete the currupted images
import PIL
list = []
for image in df['images']:
  try:
    img = PIL.Image.open(image)
  except:
    list.append(image)
list

In [ ]:
# delete df files
df = df[df['images']!= 'PetImages/Dog/11702.jpg']
df = df[df['images']!= 'PetImages/Cat/666.jpg']
len(df)

In [ ]:
df['label'] = df['label'].astype('str')
df.head()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
# Load the trained binary classification model (cat vs. dog)
model = load_model('final_model.keras')

In [ ]:
# Step 1: Load and split the dataset

# df = pd.read_csv('your_dataset.csv')

train, test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# Step 2: Create image data generators

train_generator = ImageDataGenerator(
    rescale=1./255, rotation_range=40, shear_range=0.2,
    zoom_range=0.2, horizontal_flip=True, fill_mode='nearest'
)
val_generator = ImageDataGenerator(rescale=1./255)

train_iterator = train_generator.flow_from_dataframe(
    train, x_col='images', y_col='label', target_size=(128, 128),
    batch_size=512, class_mode='binary'
)
val_iterator = val_generator.flow_from_dataframe(
    test, x_col='images', y_col='label', target_size=(128, 128),
    batch_size=512, class_mode='binary'
)

In [ ]:
# Step 3: Build the model
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPool2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPool2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPool2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# Step 4: Define callbacks
checkpoint_callback = ModelCheckpoint(
    filepath='best_model.keras', monitor='val_accuracy',
    save_best_only=True, mode='max', verbose=1
)
early_stopping_callback = EarlyStopping(
    monitor='val_accuracy', patience=5, mode='max', restore_best_weights=True
)

In [ ]:
# Step 5: Train the model
history = model.fit(
    train_iterator, epochs=15, validation_data=val_iterator,
    callbacks=[checkpoint_callback, early_stopping_callback]
)

In [ ]:
# Step 6: Save the final model
model.save('final_model.keras')
print("Final model saved as 'final_model.keras'.")

In [ ]:
# Step 7: Evaluate the model
test_loss, test_accuracy = model.evaluate(val_iterator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [6]:
import matplotlib.pyplot as plt

# Extract accuracy and loss values from history
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))  # Number of epochs

# Plot Training & Validation Accuracy
plt.figure(figsize=(8, 5))
plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title("Training vs Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)

# Plot Training & Validation Loss
plt.figure(figsize=(8, 5))
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title("Training vs Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)

# Show plots
plt.show()

In [ ]:
# !pip install mtcnn
# !pip install opencv-python
# !pip install tensorflow
# !pip install --upgrade setuptools
# !pip install --upgrade backports.tarfile

from mtcnn import MTCNN

# Test MTCNN
detector = MTCNN()
print("MTCNN loaded successfully!")

In [ ]:
# Now i am going to deploy my img classification model using Gradio library
# !pip install gradio

In [ ]:
import gradio as gr
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
from mtcnn import MTCNN  # For human face detection

In [ ]:
# Load the trained binary classification model (cat vs. dog)
model = load_model('final_model.keras')

# Initialize MTCNN for human face detection
face_detector = MTCNN()

In [ ]:
# Function to detect human faces
def detect_human_face(image):
    # Convert PIL image to numpy array
    image_array = np.array(image)
    # Detect faces using MTCNN
    faces = face_detector.detect_faces(image_array)
    return len(faces) > 0  # Return True if at least one face is detected

In [ ]:
# Function to classify cats and dogs
def classify_cat_dog(image):
    # Preprocess the image for the binary classification model
    image = image.resize((128, 128))  # Resize to match model input size
    image = np.array(image)  # Convert to numpy array
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    image = image / 255.0  # Normalize pixel values

    # Perform model inference
    predictions = model.predict(image)
    dog_prob = predictions[0][0]  # Probability of being a dog
    cat_prob = 1 - dog_prob  # Probability of being a cat

    # Determine the classification result
    if dog_prob > 0.5:
        return f"A Dog (Confidence: {dog_prob:.2f})"
    else:
        return f"A Cat (Confidence: {cat_prob:.2f})"

In [ ]:
# Main function to handle image classification
def classify_image(image):
    # Convert Gradio input image to PIL format
    image = Image.fromarray(image.astype('uint8'))

    # Step 1: Check for human faces
    if detect_human_face(image):
        return "I'm not trained to classify human faces."

    # Step 2: Classify the image as either a cat or a dog
    return classify_cat_dog(image)

In [ ]:
# Create a Gradio interface
iface = gr.Interface(
    fn=classify_image,  # Function to call
    inputs="image",     # Input type
    outputs="text",     # Output type
    title="Image Classification",  # Title of the interface
    description="Upload an image, and the model will classify it as Cat, Dog, or Human."
)

# Launch the interface
iface.launch()